# Construct Sequences to be Compared with Stata Approach 

Put prebirth year and pre-1978 part together

In [1]:
#setup
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from collections import defaultdict
from operator import itemgetter
from datetime import datetime

# set some nicer defaults for matplotlib
from matplotlib import rcParams

#these colors come from colorbrewer2.org. Each is an RGB triplet
dark2_colors = [(0.10588235294117647, 0.6196078431372549, 0.4666666666666667),
                (0.8509803921568627, 0.37254901960784315, 0.00784313725490196),
                (0.4588235294117647, 0.4392156862745098, 0.7019607843137254),
                (0.9058823529411765, 0.1607843137254902, 0.5411764705882353),
                (0.4, 0.6509803921568628, 0.11764705882352941),
                (0.9019607843137255, 0.6705882352941176, 0.00784313725490196),
                (0.6509803921568628, 0.4627450980392157, 0.11372549019607843),
                (0.4, 0.4, 0.4)]

rcParams['figure.figsize'] = (10, 6)
rcParams['figure.dpi'] = 150
rcParams['axes.color_cycle'] = dark2_colors
rcParams['lines.linewidth'] = 2
rcParams['axes.grid'] = False
rcParams['axes.facecolor'] = 'white'
rcParams['font.size'] = 14
rcParams['patch.edgecolor'] = 'none'

def remove_border(axes=None, top=False, right=False, left=True, bottom=True):
    """
    Minimize chartjunk by stripping out unnecessary plot borders and axis ticks
    
    The top/right/left/bottom keywords toggle whether the corresponding plot border is drawn
    """
    ax = axes or plt.gca()
    ax.spines['top'].set_visible(top)
    ax.spines['right'].set_visible(right)
    ax.spines['left'].set_visible(left)
    ax.spines['bottom'].set_visible(bottom)
    
    #turn off all ticks
    ax.yaxis.set_ticks_position('none')
    ax.xaxis.set_ticks_position('none')
    
    #now re-enable visibles
    if top:
        ax.xaxis.tick_top()
    if bottom:
        ax.xaxis.tick_bottom()
    if left:
        ax.yaxis.tick_left()
    if right:
        ax.yaxis.tick_right()

/Users/xzhuo/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


## Week-to-week crosswalk

Create two functions to convert between week number and calendar year and month

In [2]:
#this is the file to switch between week number in NLSY79 and calendar year, month, and day
xl_file = pd.ExcelFile('../data/continuous_week_crosswalk_2012.xlsx') 
weekdf = xl_file.parse('weekdates1')
weekdf.head()

,Week Start: Month,Week Start: Day,Week Start: Year,Calendar Year Week Number,Continuous Week Number
0,1,1,1978,1,1
1,1,8,1978,2,2
2,1,15,1978,3,3
3,1,22,1978,4,4
4,1,29,1978,5,5


In [3]:
def get_week_num(year, month, which, df=weekdf):
    """
    Given a calendar year and month, return the corresponding week number in NLSY79.
    
    If which=='first', return the first week in that month.
    If which=='last', return the last week in that month. 
    """
    
    the_month = weekdf[(weekdf['Week Start:\nMonth']==month)&(weekdf['Week Start: \nYear']==year)]
    
    if which == 'first':
        return int(the_month.iloc[0]['Continuous \nWeek Number'])
    elif which == 'last':
        return int(the_month.iloc[len(the_month)-1]['Continuous \nWeek Number'])
    
    
def get_year_month(weeknum, df=weekdf):
    """
    Given a week number, return the corresponding calendar month and year (yyyy, m).
    """
    
    the_week = weekdf[weekdf['Continuous \nWeek Number']==weeknum]
    return int(the_week.iloc[0]['Week Start: \nYear']), int(the_week.iloc[0]['Week Start:\nMonth'])    

## Retrieve analytic sample

In [4]:
# get main 18-year-seq sample
monthly_df_main = pd.read_csv('../data/monthly_df_for_testing2.csv') 
monthly_df_main.head()

,caseid_1979,month1,month2,month3,month4,month5,month6,month7,month8,month9,...,month219,month220,month221,month222,month223,month224,month225,month226,month227,month228
0,2,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1,3,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,4,4,4,4,4,4,4,4,4,4,...,3,3,3,3,3,3,3,0,0,0
3,8,4,4,4,4,4,4,4,4,4,...,1,1,1,1,1,1,1,1,1,1
4,16,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [6]:
# cut to 14-year (168 months)
monthly_df_main = monthly_df_main[['caseid_1979']+['month%i'%i for i in range(13, 181)]]
monthly_df_main.rename(columns={'month%i'%i: 'month%i'%(i-12) for i in range(13, 181)}, inplace=True)
monthly_df_main.head()

/Users/xzhuo/anaconda/lib/python2.7/site-packages/pandas/core/frame.py:2754: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


,caseid_1979,month1,month2,month3,month4,month5,month6,month7,month8,month9,...,month159,month160,month161,month162,month163,month164,month165,month166,month167,month168
0,2,5,1,4,4,4,4,4,4,4,...,3,3,3,3,3,3,3,3,3,3
1,3,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
2,4,4,4,4,4,4,4,4,4,4,...,1,1,1,1,1,1,1,1,1,1
3,8,4,4,4,4,4,4,4,4,4,...,1,1,1,1,1,1,1,1,1,1
4,16,1,1,5,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [8]:
analytic_df = pd.read_csv('../data/analytic_df_seq14.csv') #includes fertility data
print analytic_df.shape

# for the following sections, only look at the respondents who are not present in the 18-year-seq sample
analytic_df = analytic_df[(~analytic_df.caseid_1979.isin(monthly_df_main.caseid_1979.values.tolist()))]
print analytic_df.shape

(3682, 37)
(248, 37)


In [12]:
# get sequence start month

# assume birth happened in last week of the month
# thus, monthly sequence starts the month after

def get_seq_start_m(row):
    if row['c1dob_m_xrnd'] < 12:
        seq_start_y = row['c1dob_y_xrnd']
        seq_start_m = row['c1dob_m_xrnd'] + 1
    else:
        seq_start_y = row['c1dob_y_xrnd'] + 1
        seq_start_m = 1
        
    return (seq_start_y, seq_start_m)


res = analytic_df.apply(get_seq_start_m, axis=1)
analytic_df['start_y'] = map(itemgetter(0), res)
analytic_df['start_m'] = map(itemgetter(1), res)
analytic_df.head()

,caseid_1979,sample_id_1979,sample_race_1979,sample_sex_1979,c1dob_m_xrnd,c1dob_y_xrnd,c2dob_m_xrnd,c2dob_y_xrnd,c3dob_m_xrnd,c3dob_y_xrnd,...,y_deceased,last_yr,attr_before35,deceased_before35,y_child18,deceased_child18,y_child14,attr_child14,start_y,start_m
13,46,5,3,2,4,1994,7,1997,-4,-4,...,NaN,2008,False,False,2012,False,2008,False,1994,5
19,62,5,3,2,8,1996,9,1998,11,2001,...,NaN,2014,False,False,2014,False,2010,False,1996,9
22,78,5,3,2,3,1989,2,1992,9,1996,...,NaN,2006,False,False,2007,False,2003,False,1989,4
36,118,5,3,2,12,1981,4,1991,-4,-4,...,NaN,1998,False,False,1999,False,1995,False,1982,1
51,159,5,3,2,9,1985,4,1987,3,1990,...,NaN,2000,False,False,2003,False,1999,False,1985,10


## Read weekly arrays and within-job gap dates

In [9]:
#read weekly array data
wkarray_df = pd.read_csv('../data/weeklyarrays.csv')
print wkarray_df.shape
print wkarray_df.columns[:20]

(12686, 3969)
Index([u'caseid_1979', u'hrs_worked_wk_num0000_xrnd',
       u'hrs_worked_wk_num0001_xrnd', u'hrs_worked_wk_num0002_xrnd',
       u'hrs_worked_wk_num0003_xrnd', u'hrs_worked_wk_num0004_xrnd',
       u'hrs_worked_wk_num0005_xrnd', u'hrs_worked_wk_num0006_xrnd',
       u'hrs_worked_wk_num0007_xrnd', u'hrs_worked_wk_num0008_xrnd',
       u'hrs_worked_wk_num0009_xrnd', u'hrs_worked_wk_num0010_xrnd',
       u'hrs_worked_wk_num0011_xrnd', u'hrs_worked_wk_num0012_xrnd',
       u'hrs_worked_wk_num0013_xrnd', u'hrs_worked_wk_num0014_xrnd',
       u'hrs_worked_wk_num0015_xrnd', u'hrs_worked_wk_num0016_xrnd',
       u'hrs_worked_wk_num0017_xrnd', u'hrs_worked_wk_num0018_xrnd'],
      dtype='object')


In [10]:
#read within-job-gap data
gap_df = pd.read_csv("../data/withingaps.csv")
print gap_df.shape
print gap_df.columns[:20]

(12686, 839)
Index([u'caseid_1979', u'per1_start_1979_job01_xrnd',
       u'per1_start_1979_job02_xrnd', u'per1_start_1979_job03_xrnd',
       u'per1_start_1979_job04_xrnd', u'per1_start_1979_job05_xrnd',
       u'per1_stop_1979_job01_xrnd', u'per1_stop_1979_job02_xrnd',
       u'per1_stop_1979_job03_xrnd', u'per1_stop_1979_job04_xrnd',
       u'per1_stop_1979_job05_xrnd', u'per2_start_1979_job01_xrnd',
       u'per2_start_1979_job02_xrnd', u'per2_start_1979_job03_xrnd',
       u'per2_start_1979_job04_xrnd', u'per2_start_1979_job05_xrnd',
       u'per2_stop_1979_job01_xrnd', u'per2_stop_1979_job02_xrnd',
       u'per2_stop_1979_job03_xrnd', u'per2_stop_1979_job04_xrnd'],
      dtype='object')


In [11]:
# drops weekly columns after 2013
# our sequences only goes up to 2013

print wkarray_df.shape # before

l = ['hrs_worked_wk_num%04d_xrnd'%i for i in range(1880, 1984)]
wkarray_df.drop(labels=l, axis=1, inplace=True)

l = ['status_wk_num%04d_xrnd'%i for i in range(1880, 1984)]
wkarray_df.drop(labels=l, axis=1, inplace=True)

print wkarray_df.shape # after

(12686, 3969)
(12686, 3761)


## Do we need to consider any pre-1978 month?

No. Only 10 respondents have pre-1978 months in their sequences, and these repsondents never worked prior to 1978/1/1. 

In [13]:
# get only respondents whose prebirth sequence proceeds 1978 - 10 cases

pre1978_df = analytic_df[analytic_df.start_y<1978]
print pre1978_df.shape

(10, 39)


In [14]:
merged = pre1978_df.merge(wkarray_df, on='caseid_1979')
merged.hrs_worked_wk_num0000_xrnd.value_counts()

0    10
Name: hrs_worked_wk_num0000_xrnd, dtype: int64

## Identify maternity leaves

Maternity leave could cover the birth, start after the birth, or end before the birth. 

Let's assume any within-job gap that *overlaps with birth or started within 4 months (16 weeks) after birth* as maternity leave. 

### Does any of our respondents have gaps that started/ended prior to 1/1/1978? 

NO

In [15]:
merged = analytic_df.merge(gap_df, on='caseid_1979') # merge datasets

In [16]:
# count how many zero values in each job gap column
# NONE

for col in merged.columns.tolist():
    if col.startswith('per'): # only check job gap columns
        count = sum(merged[col]==0)
        if count > 0:
            print col, count

In [17]:
###modified to handle months prior to 1978


def get_gaps(row):
    """
    Given a row of respondent data, 
    Return a list of tuples (start week, end week) of work gaps for respondent i
    """
    
    gap_row = gap_df[gap_df.caseid_1979==row['caseid_1979']].iloc[0]
    gaps = []
    
    years = range(1979, 1995)+range(1996, 2015, 2)
    
    #look for within-job gaps in all years. sometimes the gaps were recorded in later years
    for y in years: 
        for j in range(1, 5): #4 possible gap periods
            for k in range(1, 6): #5 possible jobs
                try: 
                    gap_start = gap_row['per%s_start_%s_job0%s_xrnd'%(j, y, k)]
                    gap_stop = gap_row['per%s_stop_%s_job0%s_xrnd'%(j, y, k)]
                    
                    if gap_start >= 0 and gap_stop >= 0: # valid, no missing
                        gaps.append((gap_start, gap_stop))

                except KeyError: # some years do not report 5 jobs
                    pass
                
    return gaps


def find_maternal_leave_no_before(row):
    """
    Given a row of respondent data, return a list of all weeks identified as maternal leaves
    """
    
    leaves = [] # store all weeks that are part of maternity leaves
    
    gaps = get_gaps(row)
    
    for j in range(1, 12): # all births
        dob_y = row['c%idob_y_xrnd'%j]
        dob_m = row['c%idob_m_xrnd'%j]
        
        if dob_y>0 and dob_m>0:
            
            # note: job gap dates do not go prior to 1978 
            # (we do not count the three cases with dates prior to 1/1/1978, as explained above)
            
            if dob_y >= 1978: 
                
                for (gap_start, gap_stop) in gaps:
                    birth_wks = range(get_week_num(dob_y, dob_m, 'first'), get_week_num(dob_y, dob_m, 'last')+1)
                    gap_wks = range(gap_start, gap_stop+1)
                    is_ma_leave = False

                    if set(birth_wks)&set(gap_wks): #the gap overlaps with the birth
                        is_ma_leave = True

                    elif gap_start>birth_wks[-1] and gap_start-birth_wks[-1]<=16: #gap started within 16 weeks after birth
                        is_ma_leave = True

                    if is_ma_leave:
                        leaves += range(gap_start, gap_stop+1) # store all weeks that are maternity leaves
    
    return leaves

## Tranform weekly sequences

My status classification scheme:

* 1 - full-time employment
* 2 - working with missing hours
* 3 - part-time employment
* 4 - nonworking 
* 5 - maternity leave


NEW: If respondents reported working 0 hours in week 0 (total hours worked pre-1978), assign 4-nonworking to all weeks before week 1. 

In [18]:
# select only week array data for our selected sample
merged_wkarray_df = analytic_df.merge(wkarray_df, on='caseid_1979')
print merged_wkarray_df.shape

(248, 3799)


In [19]:
### modification: 
# (1) assign weeks with labor force status 'w' and work hour 0 to part-time
# (2) include prebirth year


# create a dictionary to store data
data = defaultdict(list)


for i, row in merged_wkarray_df.iterrows(): #iterate through weekly array rows

    #keep track of progress
    if i%100 == 0: 
        print i,
    
    #keep three columns: id, start_y, start_m
    data['caseid_1979'].append(row['caseid_1979'])
    data['start_y'].append(row['start_y'])
    data['start_m'].append(row['start_m'])

        
    #maternal leaves
    leaves = find_maternal_leave_no_before(row)
        
    for week_num in range(1, 1880): #iterate through all weekly work variables
         
        #if the week is maternity leave
        if week_num in leaves:
            data['week%s'%week_num].append(5) 
            continue
        
        #otherwise:
        my_status = None
        
        #get the hours worked this week 
        hours = row['hrs_worked_wk_num%04d_xrnd'%week_num]
    
        #get the employment status this week
        status = row['status_wk_num%04d_xrnd'%week_num]

        if (status >= 100 and status <= 2615):
            status = 'w' #working
        
        
        ### define employment status according to my classification scheme ###
        
        # missing
        if hours == 0 and status == 0:
            my_status = 0
            
        # full time
        elif hours >= 35 and status in ['w', 3]:
            my_status = 1
        elif hours == 0 and status == 7:
            my_status = 1
            
        # working, missing hours
        elif hours < 0 and status in ['w', 3]: 
            my_status = 2
            
        # part time
        elif hours >= 0 and hours < 35 and status in ['w', 3]:
            my_status = 3
            
        # non-working
        elif hours <= 0 and status in [2, 4, 5]:
            my_status = 4
            
        ### NEW ###
        elif hours >= 0 and hours < 35 and status in [4, 5]:
            my_status = 4
            print row['caseid_1979'], hours, status

        else:
            raise Exception, 'unknown hours and status: (%s, %s)'%(hours, status)
            
        data['week%s'%week_num].append(my_status) #make the week number start from 0, easier for indexing in Python

0 2030 3 5
100 200


In [20]:
#create the new weekly array dataframe with my edited employment status

#week number range: 1 - 1879

#order the columns in the new dataframe
cols = ['caseid_1979', 'start_y', 'start_m'] + ['week%s'%n for n in range(1, 1880)]

full_df = pd.DataFrame(data, columns=cols)
print full_df.iloc[0]

caseid_1979      46
start_y        1994
start_m           5
week1             4
week2             4
week3             4
week4             4
week5             4
week6             4
week7             4
week8             4
week9             4
week10            4
week11            4
week12            4
week13            4
week14            4
week15            4
week16            4
week17            4
week18            4
week19            4
week20            4
week21            4
week22            4
week23            4
week24            4
week25            4
week26            4
week27            4
               ... 
week1850          0
week1851          0
week1852          0
week1853          0
week1854          0
week1855          0
week1856          0
week1857          0
week1858          0
week1859          0
week1860          0
week1861          0
week1862          0
week1863          0
week1864          0
week1865          0
week1866          0
week1867          0
week1868          0


## Construct 14 year monthly sequences

In [21]:
def find_modal(l):
    """
    Given a list l, return the most common element in the list. 
    if there is a tie, return all elements.
    """
    
    #first, count frequency of each unique item in the list
    counter = defaultdict(int)
    for item in l:
        counter[item] += 1

    #rank items from most frequent to least frequent
    popular_items = sorted(counter, key=counter.get, reverse = True)
    
    most_common = [popular_items[0]]
    
    for other in popular_items[1:]:
        if counter[other] < counter[most_common[0]]:
            break
        elif counter[other] == counter[most_common[0]]: #find an equally frequent item
            most_common.append(other)
        else:
            print 'impossible'

    return most_common

#for example
print find_modal([1, 1, 2, 1, 5])
print find_modal([1, 1, 2, 1, 5, 2, 2, 33])

[1]
[1, 2]


In [22]:
import random
random.seed(7222014)

res_data = defaultdict(list) #create an empty dictionary to store data

count_ties = defaultdict(list)
combn_ties = defaultdict(int)


for i, row in full_df.iterrows():
    
    # keep track of progress
    if i%100 == 0:
        print i, 
    
    # keep basic info: id, sequence start year and month
    res_data['caseid_1979'].append(row['caseid_1979'])
    res_data['start_y'].append(row['start_y'])
    res_data['start_m'].append(row['start_m'])
    
    
    y = row['start_y']
    m = row['start_m']

    
    # look at weekly statuses
    for j in range(1, 169): # from month 1 to month 168, 14 years
        
        if y < 1978: #prior to 1978
            hrs_pre1978 = merged_wkarray_df[merged_wkarray_df.caseid_1979==row['caseid_1979']].iloc[0]['hrs_worked_wk_num0000_xrnd']
            
            if hrs_pre1978 == 0: # never worked before 1978
                res_data['month%s'%j].append(4) # assign 4 - nonemployed
            else:
                raise Exception, 'should not have any pre-1978 working months'
            
        else:
            start = get_week_num(y, m, 'first')
            end = get_week_num(y, m, 'last')
            wk_statuses = [row['week%i'%k] for k in range(start, end+1)]

            #find unique items in the given month 
            distinct = list(set(wk_statuses))
            distinct.sort()

            if len(distinct) == 1:
                res_data['month%s'%j].append(distinct[0])

            elif len(distinct) >= 2:
                modal = find_modal(wk_statuses)
                if len(modal) == 1:
                    res_data['month%s'%j].append(modal[0])
                else:
                    #res_data['month%s'%j].append(random.choice(modal))
                    ordered = [1, 4, 3, 5, 2, 0] # FT, not working, PT, mat leave, working miss hours, total miss
                    modal_indx = [ordered.index(val) for val in modal]
                    res_data['month%s'%j].append(ordered[min(modal_indx)])
                    
                    # count and record
                    count_ties[row['caseid_1979']].append(j)
                    
                    modal.sort()
                    modal = [str(val) for val in modal]
                    combn_ties[','.join(modal)] += 1
                    
                    

         
        #move to next month
        if m < 12:
            m += 1
        elif m == 12:
            m = 1
            y += 1

            
cols = ['caseid_1979', 'start_y', 'start_m'] + ['month%s'%i for i in range(1, 169)]
res_df = pd.DataFrame(res_data, columns=cols) #restricted dataframe

0 100 200


In [23]:
res_df.head()

,caseid_1979,start_y,start_m,month1,month2,month3,month4,month5,month6,month7,...,month159,month160,month161,month162,month163,month164,month165,month166,month167,month168
0,46,1994,5,1,1,3,3,3,3,3,...,1,1,1,1,1,1,1,1,0,0
1,62,1996,9,5,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,78,1989,4,1,4,4,4,4,3,3,...,4,4,4,4,4,4,4,4,3,3
3,118,1982,1,4,4,4,4,3,3,3,...,1,1,1,1,1,1,1,1,1,1
4,159,1985,10,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [27]:
# how many ties?
print '%s respondents had ties'%len(count_ties.keys())

lens_per_r = [len(v) for v in count_ties.values()]
print '%s respondent-month pairs had ties'%sum(lens_per_r)

113 respondents had ties
236 respondent-month pairs had ties


In [26]:
# combination of ties
print combn_ties

defaultdict(<type 'int'>, {'0,4': 3, '0,3': 6, '0,1': 4, '3,4': 65, '3,5': 8, '1,4': 90, '1,5': 17, '1,3': 40, '2,4': 2, '2,3': 1})


## Output dataset

In [28]:
print res_df.shape
res_df = pd.concat([monthly_df_main, res_df])
print res_df.shape

(248, 171)
(3682, 171)


In [30]:
res_df = res_df[['caseid_1979']+['month%i'%i for i in range(1, 169)]]
res_df.to_csv('../data/monthly_df_for_testing_seq14.csv', index=False)

In [8]:
# res_df = pd.read_csv('../data/monthly_df_for_testing_seq14.csv')